In [1]:
pip install numpy pandas scikit-learn

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/eb/cb/22891a0048dcb66c61a7fb3153829b1b9e09d6242e4dae00b17983377340/scikit_learn-1.4.0-cp39-cp39-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/81/12/fd4dea011af9d69e1cad05c75f3f7202cdcbeac9b712eea58ca779a72865/threadpoolctl-3.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 8.8 MB/s eta 0:00:0000:01m0:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 30.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2 -> 23.3.2
[notice] To update, run: /Applications/Xcode.ap

In [2]:
# Data Loading and Preprocessing
import pandas as pd
import glob

# Path to your folder
path = 'labled_data'

# Find all CSV files in the folder
all_files = glob.glob(path + "/Tracking_video*_labled.csv")

# List to store the dataframes
li = []

# Read each CSV file and add it to the list
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# Combine all dataframes in the list into a single dataframe
data = pd.concat(li, axis=0, ignore_index=True)

In [3]:
# Model Training
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Features
X = data.drop('Label', axis=1)
# Labels
y = data['Label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = RandomForestClassifier()

# Train the model
model.fit(X_train, y_train)

RandomForestClassifier()

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

# Features
X = data.drop('Label', axis=1)
# Labels
y = data['Label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use mean imputation for missing values
imputer = SimpleImputer(strategy='mean')

# Fit on the training data and transform both training and test data
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Initialize the models
models = [
    LogisticRegression(),
    SVC(),
    DecisionTreeClassifier(),
    GradientBoostingClassifier(),
    KNeighborsClassifier(),
    MLPClassifier(max_iter=1000)  # Increase max_iter for larger datasets
]

# Train and evaluate each model
for model in models:
    model_name = model.__class__.__name__
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {accuracy}")

/Users/jonasgwozdz/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression Accuracy: 0.6014259891544487
SVC Accuracy: 0.46294436633862224
DecisionTreeClassifier Accuracy: 0.941654950793332
GradientBoostingClassifier Accuracy: 0.8379192608957622
KNeighborsClassifier Accuracy: 0.8186382807792729
MLPClassifier Accuracy: 0.48132155051215103


In [13]:
import joblib

# Train and evaluate each model
for model in models:
    model_name = model.__class__.__name__

    # Save the model
    joblib.dump(model, f'{model_name}.pkl')

In [22]:
# Model Evaluation
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Load the DecisionTreeClassifier model
model = joblib.load('DecisionTreeClassifier.pkl')

# Predict the labels for the test set
y_pred = model.predict(X_test)

# Print the accuracy and the classification report
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.941654950793332
              precision    recall  f1-score   support

   Ascending       0.91      0.91      0.91      2367
  Descending       0.93      0.93      0.93      2981
       Pause       0.97      0.97      0.97      4610

    accuracy                           0.94      9958
   macro avg       0.93      0.93      0.93      9958
weighted avg       0.94      0.94      0.94      9958



In [5]:
# Model Saving
import joblib

# Save the model
joblib.dump(model, 'model.pkl')

['model.pkl']

In [ ]:
# Model Loading and Usage
import joblib

# Load the model
loaded_model = joblib.load('DecisionTreeClassifier.pkl')

# Use the loaded model to make predictions
# For example, predict the label for the first instance in the test set
single_prediction = loaded_model.predict([X_test.iloc[0]])
print("Prediction for the first instance in the test set:", single_prediction)

In [21]:
import pandas as pd
# Model Loading and Usage
import joblib

# Load the model
loaded_model = joblib.load('DecisionTreeClassifier.pkl')
# Load new data
new_data = pd.read_csv('new_data.csv')

# If the new data includes the 'Label' column, drop it
if 'Label' in new_data.columns:
    new_data = new_data.drop('Label', axis=1)

# Use the loaded model to make predictions
predictions = loaded_model.predict(new_data)

# Add the predictions back into the 'Label' column
new_data['Label'] = predictions

# Save the DataFrame with the new labels back to a CSV file
new_data.to_csv('new_data_with_predictions.csv', index=False)

/Users/jonasgwozdz/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
